In [10]:
from dotenv import load_dotenv
load_dotenv()

import os
import asyncio
from postchain_client_py import BlockchainClient
from postchain_client_py.blockchain_client.types import NetworkSettings, Operation, Transaction

# Initialize network settings
settings = NetworkSettings(
    node_url_pool=["http://localhost:7740"],
    blockchain_iid=0,
)
    
# Create client and execute query
client = await BlockchainClient.create(settings)

In [11]:
result = await client.query("get_all_data")
print(f"Telemetry Data ({len(result)} records):")
for i, record in enumerate(result, 1):
    sensor_id, battery, timestamp = record
    print(f"  {i}. Sensor: {sensor_id}, Battery: {battery}%, Timestamp: {timestamp}")

Telemetry Data (0 records):


In [12]:
from coincurve import PrivateKey

private_bytes = bytes.fromhex(os.getenv("PRIV_KEY"))
private_key = PrivateKey(private_bytes)
public_key = private_key.public_key.format(compressed=True)
print(public_key.hex())

03922b871a76a243957bdc26f1204c7612f221c9213697afef17a9ac602b84a042


In [13]:
operation = Operation(
    op_name="add_data",
    args=[
        "ChromBot",
        "69"
    ]
)

print(operation)

Operation(op_name='add_data', args=['ChromBot', '69'])


In [14]:
transaction = Transaction(
    operations=[operation],
    signers=[public_key],
    signatures=None,
    blockchain_rid=client.config.blockchain_rid
)
print(transaction)

Transaction(operations=[Operation(op_name='add_data', args=['ChromBot', '69'])], signers=[b'\x03\x92+\x87\x1av\xa2C\x95{\xdc&\xf1 Lv\x12\xf2!\xc9!6\x97\xaf\xef\x17\xa9\xac`+\x84\xa0B'], signatures=None, blockchain_rid='099C14C931139955BB7651CE550BFB89112A02B239BF503359BDA6D8D46856B6')


In [15]:
signed_tx = await client.sign_transaction(transaction, private_bytes)
receipt = await client.send_transaction(signed_tx, do_status_polling=True)

print(receipt)   

TransactionReceipt(status='confirmed', status_code=200, transaction_rid=b"^H9\xcd\xfdM\x14\xda\xb0\xca|#\xc5y\xcc'\xe2\xcd\x078\x90k_a\x1b.\xd7H\xbc\xa5M\xb9", message=None, cluster_anchored_tx=None, system_anchored_tx=None)


In [20]:
result = await client.query("get_all_data")
print(f"Telemetry Data ({len(result)} records):")
for i, record in enumerate(result, 1):
    sensor_id = record['sensor_id']
    battery = record['battery']
    timestamp = record['timestamp']
    print(f"  {i}. Sensor: {sensor_id}, Battery: {battery}%, Timestamp: {timestamp}")

Telemetry Data (1 records):
  1. Sensor: ChromBot, Battery: 69%, Timestamp: 1758693993198


In [16]:
# Clean up
await client.rest_client.close()